## Exercise 2

We'll first just copy relevant code from the last exercise to start where we left off.

In [1]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

# Relevant code from exercise 1.1
product_data = pd.read_csv('https://github.com/Mixtape-Sessions/Demand-Estimation/raw/main/Exercises/Data/products.csv')

# Relevant code from exercise 1.2
product_data['market_size'] = product_data['city_population'] * 90
product_data['market_share'] = product_data['servings_sold'] / product_data['market_size']

# Relevant code from exercise 1.4
product_data = product_data.rename(columns={
    'market': 'market_ids',
    'product': 'product_ids',
    'market_share': 'shares',
    'price_per_serving': 'prices',
})

# Relevant code from exercise 1.6
first_stage = smf.ols('prices ~ 0 + price_instrument + C(market_ids) + C(product_ids)', product_data)
first_stage_results = first_stage.fit(cov_type='HC0')
product_data = product_data.rename(columns={'price_instrument': 'demand_instruments0'})
iv_problem = pyblp.Problem(pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), product_data)
iv_results = iv_problem.solve(method='1s')

# Relevant code from exercise 1.7
counterfactual_market = 'C01Q2'
counterfactual_data = product_data.loc[product_data['market_ids'] == counterfactual_market, ['product_ids', 'mushy', 'prices', 'shares']]
counterfactual_data['new_prices'] = counterfactual_data['prices']
counterfactual_data.loc[counterfactual_data['product_ids'] == 'F1B04', 'new_prices'] /= 2
counterfactual_data['new_shares'] = iv_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['iv_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']

### 1. Describe cross-market variation

Let's first look at the amount of cross-market choice set variation.

In [2]:
choice_variation = product_data.groupby('market_ids', as_index=False).agg(**{
    'products': ('product_ids', 'count'),
    'mushy_mean': ('mushy', 'mean'),
    'mushy_std': ('mushy', 'std'),
    'prices_mean': ('prices', 'mean'),
    'prices_std': ('prices', 'std'),
})
choice_variation.describe()

,products,mushy_mean,mushy_std,prices_mean,prices_std
count,94.0,9.400e+01,9.400e+01,94.000,94.000
mean,24.0,3.333e-01,4.815e-01,0.126,0.029
std,0.0,4.465e-16,3.349e-16,0.005,0.004
min,24.0,3.333e-01,4.815e-01,0.112,0.022
25%,24.0,3.333e-01,4.815e-01,0.122,0.026
50%,24.0,3.333e-01,4.815e-01,0.126,0.028
75%,24.0,3.333e-01,4.815e-01,0.129,0.031
max,24.0,3.333e-01,4.815e-01,0.138,0.038


The same 20 products are present in each market, and mushyness does not change across markets. So there is no cross-market variation in the number of products or mushyness. However, there is cross-market variation in prices. Recalling the linear regression intuition for identification, this means that we have hope of credibly identifying the amount of unobserved preference heterogeneity for price, but not for a constant or mushy.

Next, let's load the new demographic data.

In [3]:
demographic_data = pd.read_csv('https://github.com/Mixtape-Sessions/Demand-Estimation/raw/main/Exercises/Data/demographics.csv')
demographic_data = demographic_data.rename(columns={'market': 'market_ids'})
demographic_data.sample(n=5, random_state=0)

,market_ids,quarterly_income
511,C18Q2,3785.817
124,C05Q1,4359.937
217,C08Q1,3822.916
1426,C47Q2,1708.163
891,C32Q1,3983.035


Income has a long right tail, so it's easier to work in logs.

In [4]:
demographic_data['log_income'] = np.log(demographic_data['quarterly_income'])
demographic_data[['quarterly_income', 'log_income']].describe()

,quarterly_income,log_income
count,1880.000,1880.000
mean,4575.000,8.091
std,3679.854,0.940
min,23.411,3.153
25%,2128.625,7.663
50%,3836.111,8.252
75%,5884.614,8.680
max,33724.436,10.426


Let's see how much cross-market demographic variation there is.

In [5]:
demographic_variation = demographic_data.groupby('market_ids', as_index=False).agg(**{
    'log_income_mean': ('log_income', 'mean'),
    'log_income_std': ('log_income', 'std'),
})
demographic_variation.describe()

,log_income_mean,log_income_std
count,94.000,94.000
mean,8.091,0.885
std,0.289,0.242
min,7.499,0.523
25%,7.872,0.710
50%,8.093,0.831
75%,8.338,1.086
max,8.622,1.539


There seems to be a good amount of cross-market demographic variation. Consumers' income varies a good amount across our markets. This means that using this variation, we have a hope of credibly estimating how income shifts the preference of different characteristics. However, because we have market fixed effects, a parameter on income alone would be collinear and not identified.

### 2. Estimate a parameter on mushy $\times$ log income

First, we need consumer type data with demographic draws, standard normal nodes (for later), and uniform weights.

In [6]:
agent_data = demographic_data[['market_ids', 'log_income']].groupby('market_ids', as_index=False).sample(n=1000, replace=True, random_state=0)
agent_data[['nodes0', 'nodes1', 'nodes2']] = np.random.default_rng(seed=0).normal(size=(len(agent_data), 3))
agent_data['weights'] = 1 / agent_data.groupby('market_ids').transform('size')
agent_data.sample(n=5, random_state=0)

,market_ids,log_income,nodes0,nodes1,nodes2,weights
1100,C37Q2,7.590,-1.401,1.667,-1.312,0.001
1860,C65Q2,7.584,-1.609,0.747,-0.582,0.001
1726,C58Q1,7.949,-0.914,1.266,0.043,0.001
491,C18Q1,8.239,-0.946,-0.069,-1.075,0.001
314,C12Q2,8.569,-1.268,-0.034,-0.462,0.001


We also need a new instrument.

In [7]:
product_data = product_data.merge(demographic_variation[['market_ids', 'log_income_mean']], on='market_ids')
product_data['demand_instruments1'] = product_data['log_income_mean'] * product_data['mushy']

To model an interaction between mushy and the log of income, we need two new formulations when setting up the problem.

In [8]:
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('0 + mushy'))
agent_formulation = pyblp.Formulation('0 + log_income')
mushy_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
mushy_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     1     1    2     2  

Formulations:
       Column Indices:             0     
-----------------------------  ----------
 X1: Linear Characteristics      prices  
X2: Nonlinear Characteristics    mushy   
       d: Demographics         log_income

Now that we have a nonlinear GMM estimator, we need a nonlinear optimizer. We'll use SciPy's trust region algorithm to optimize the objective. The default tolerances of ``1e-8`` will work fine for our setting, but it's good practice to explicitly set them yourself.

In [9]:
optimization = pyblp.Optimization('trust-constr', {'gtol': 1e-8, 'xtol': 1e-8})

Turning PyBLP verbosity on before optimization and off afterwards lets us monitor optimization progress in real time. We aren't adding a parameter in $\Sigma$ so we'll set that to zero (zeros mean that it will remain fixed during optimization) and we'll choose an arbitrary starting point of 1 for our new parameter in $\Pi$.

In [10]:
pyblp.options.verbose = True
mushy_results = mushy_problem.solve(sigma=0, pi=1, method='1s', optimization=optimization)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy    |   Pi:   log_income
------  ---------  |  -----  ----------
mushy   +0.00E+00  |  mushy  +1.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy    |   Pi:   log_income
------  ---------  |  -----  ----------
mushy   +0.00E+00  |  mushy    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy    |   Pi:   log_income
------  ---------  |  -----  ----------
mushy   +0.00E+00  |  mushy    +INF    

Starting optimization ...


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point 

After seeing "marching down the gradient" behavior, the gradient norm is near-zero at the optimum, indicating the optimization was successful. Since the model is just identified, we also see a near-zero objective. Finally, the Hessian is positive, indicating the second order conditions are satisfied.

The new parameter is positive, indicating that wealthier individuals prefer mushy cereal. Dividing it by $\hat{\alpha}$ gives 0.25 / 30.6 = $0.01 more willingness to pay for a mushy cereal for every 1% increase in income (recall that a log regressor means we can interpret in terms of percents).

### 3. Make sure you get the same estimate with random starting values

Let's set some bounds for the new parameter and optimize from three random starting values within these bounds.

In [11]:
pi_bounds = (-10, +10)
for seed in range(3):
    initial_pi = np.random.default_rng(seed).uniform(*pi_bounds)
    seed_results = mushy_problem.solve(sigma=0, pi=initial_pi, pi_bounds=pi_bounds, method='1s', optimization=optimization)
    print(f"Initial: {initial_pi}. Estimated: {seed_results.pi[0, 0]}.")

Initial: 2.739233746429086. Estimated: 0.25128322135971515.
Initial: 0.23643249400513433. Estimated: 0.2512832213611108.
Initial: -4.767757315013672. Estimated: 0.2512832207536896.


We get the same estimate each time, indicating (along with all our other diagnostics) that optimization worked well and our estimate is indeed the global minimum.

### 4. Evaluate changes to the price cut counterfactual

Let's re-run the price cut counterfactual from last exercise and compare.

In [12]:
counterfactual_data['new_shares'] = mushy_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['mushy_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,mushy,prices,shares,new_prices,new_shares,iv_change,mushy_change
24,F1B04,1,0.078,6.443e-03,0.039,2.084e-02,223.638,223.522
25,F1B06,1,0.141,1.413e-01,0.141,1.392e-01,-1.450,-1.478
26,F1B07,1,0.073,8.789e-02,0.073,8.660e-02,-1.450,-1.478
27,F1B09,0,0.077,6.621e-03,0.077,6.526e-03,-1.450,-1.438
28,F1B11,0,0.167,5.427e-02,0.167,5.349e-02,-1.450,-1.438
29,F1B13,0,0.092,2.198e-02,0.092,2.166e-02,-1.450,-1.438
30,F1B17,1,0.154,1.055e-02,0.154,1.039e-02,-1.450,-1.478
31,F1B30,0,0.150,1.310e-03,0.150,1.291e-03,-1.450,-1.438
32,F1B45,0,0.147,1.052e-02,0.147,1.037e-02,-1.450,-1.438
33,F2B05,0,0.099,5.907e-02,0.099,5.822e-02,-1.450,-1.438


Results are fairly similar, which is unsurprising because the new parameter is estimated to be fairly small. Substitution from other products (and hence cannibalization) seems a bit more reasonable in the sense that we see a bit more substitution from more similar (along the mushy dimension) producucts than others. This is because high-income consumers are more likely to substitute to the now-cheaper mushy product, so products with more high-income consumers (i.e. mushy ones) are more likely to have strong reductions in demand due to substitution.

### 5. Estimate parameters on price $\times$ log income and unobserved preferences

We'll first compute the fitted values from our prive IV's first stage and verify that they are strongly correlated with prices.

In [13]:
product_data['predicted_prices'] = first_stage_results.fittedvalues
product_data[['prices', 'predicted_prices']].corr()

,prices,predicted_prices
prices,1.000,0.982
predicted_prices,0.982,1.000


Next, we'll construct an IV to target the new parameter in $\Pi$.

In [14]:
product_data['demand_instruments2'] = product_data['log_income_mean'] * product_data['predicted_prices']

We'll also construct an IV to target the new parameter in $\Sigma$.

In [15]:
compute_differentiation = lambda x: np.sum((x.values[:, None] - x.values[None, :])**2, axis=1)
product_data['demand_instruments3'] = product_data.groupby('market_ids')['predicted_prices'].transform(compute_differentiation)

Let's initialize a new problem with the new instruments.

In [16]:
product_formulations = (pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), pyblp.Formulation('0 + mushy + prices'))
agent_formulation = pyblp.Formulation('0 + log_income')
rc_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
rc_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     2     1    4     2  

Formulations:
       Column Indices:             0         1   
-----------------------------  ----------  ------
 X1: Linear Characteristics      prices          
X2: Nonlinear Characteristics    mushy     prices
       d: Demographics         log_income        

Let's solve the problem, choosing a starting value for the old parameter close to its estimate and some arbitrary starting values for the two new parameters.

In [17]:
pyblp.options.verbose = True
rc_results = rc_problem.solve(
    sigma=[
        [0, 0],
        [0, 1],
    ], 
    pi=[
        [0.2],
        [1],
    ], 
    method='1s', 
    optimization=optimization,
)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy   +2.00E-01 
prices  +0.00E+00  +1.00E+00  |  prices  +1.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.00E+00  |  prices    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     +INF    
prices  +0.00E+00    +INF     |  prices    +INF    

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective     Projected                                   
Step     Time       Iterations   Evaluations  Iterations   

Again, the near-zero objective and first- and second-order condition tests indicate that we've reached a global minimum. To save time in this exercise, we won't try multiple starting values.

With an average log income of 8.09 from above, the average price coefficient is 13.7 - 5.96 * 8.09 = -34.5, which is fairly close to our old non-random price coefficient estimate $\hat{\alpha}$ of -30.6. With a standard deviation of log income of 0.9, close to the standard deviation of our $N(0, 1)$ draws, there seems to be about as much unobserved price sensitivity heterogeneity as heterogeneity from income (the two new parameter estimates are about the same magnitude). For more precise interpretations of our new parameters, we could compute elasticities for different consumer types and compare.

### 6. Evaluate changes to the price counterfactual

Let's re-run the price cut counterfactual and compare.

In [18]:
counterfactual_data['new_shares'] = rc_results.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['rc_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,mushy,prices,shares,new_prices,new_shares,iv_change,mushy_change,rc_change
24,F1B04,1,0.078,6.443e-03,0.039,2.481e-02,223.638,223.522,285.128
25,F1B06,1,0.141,1.413e-01,0.141,1.390e-01,-1.450,-1.478,-1.621
26,F1B07,1,0.073,8.789e-02,0.073,8.627e-02,-1.450,-1.478,-1.850
27,F1B09,0,0.077,6.621e-03,0.077,6.502e-03,-1.450,-1.438,-1.808
28,F1B11,0,0.167,5.427e-02,0.167,5.346e-02,-1.450,-1.438,-1.496
29,F1B13,0,0.092,2.198e-02,0.092,2.159e-02,-1.450,-1.438,-1.759
30,F1B17,1,0.154,1.055e-02,0.154,1.038e-02,-1.450,-1.478,-1.575
31,F1B30,0,0.150,1.310e-03,0.150,1.289e-03,-1.450,-1.438,-1.556
32,F1B45,0,0.147,1.052e-02,0.147,1.036e-02,-1.450,-1.438,-1.568
33,F2B05,0,0.099,5.907e-02,0.099,5.804e-02,-1.450,-1.438,-1.735


A higher increase in demand for the product with the price cut is in-line with a slightly more negative coefficient on price. Adding a substantial amount of preference heterogeneity for price also resulted in much more reasonable substitution (and hence cannibalization) along the price dimension, in the sense that cereals with a similar price per serving as the changed one had more demand reductions than others. However, there is still very limited substitution within the mushy dimension because with no cross-market mushy variation, we couldn't credibly identify a parameter in $\Sigma$ on mushy.

## Supplemental Questions

### 1. Use different numbers of Monte Carlo draws

For convenience, let's first define a function for initializing and solving a problem for a given set of agent data.

In [19]:
def solve(new_specification, new_agent_data):
    new_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, new_agent_data)
    new_results = new_problem.solve(
        sigma=[
            [0, 0],
            [0, 6],
        ], 
        pi=[
            [0.2],
            [-6],
        ], 
        method='1s', 
        optimization=optimization,
    )
    return {
        "Specification": new_specification,
        "Time": new_results.optimization_time,
        "Converged": new_results.converged,
        "Gradient Norm": new_results.projected_gradient_norm,
        "Sigma on Price": new_results.sigma[1, 1],
        "Pi on Mushy": new_results.pi[0, 0],
        "Pi on Price": new_results.pi[1, 0],
        "Alpha": new_results.beta[0, 0],
    }

Now let's try using different numbers of Monte Carlo draws. For this exercise we only need one set of nodes.

In [20]:
agent_results = []
for draws in [10, 100, 500, 1_000, 2_000]:
    print(f"Using {draws} ...")
    mc_data = demographic_data[['market_ids', 'log_income']].groupby('market_ids', as_index=False).sample(n=draws, replace=True, random_state=0)
    mc_data['nodes0'] = np.random.default_rng(seed=0).normal(size=len(mc_data))
    mc_data['weights'] = 1 / draws
    agent_results.append(solve(f"{draws} Monte Carlo Draws", mc_data))

pd.DataFrame(agent_results)

Using 10 ...
Using 100 ...
Using 500 ...
Using 1000 ...
Using 2000 ...


,Specification,Time,Converged,Gradient Norm,Sigma on Price,Pi on Mushy,Pi on Price,Alpha
0,10 Monte Carlo Draws,2.961,True,1.425e-09,7.271,0.127,-7.085,22.752
1,100 Monte Carlo Draws,3.604,True,1.123e-06,5.748,0.104,-5.728,12.020
2,500 Monte Carlo Draws,8.926,True,8.619e-09,6.064,0.108,-6.054,14.487
3,1000 Monte Carlo Draws,13.725,True,2.120e-07,6.141,0.109,-5.956,13.664
4,2000 Monte Carlo Draws,18.678,True,2.116e-07,6.051,0.108,-5.989,13.971


Estimates seem to stabilize by 1,000 draws, but there are some clear differences before then, suggesting that only a small number of Monte Carlo draws aren't enough to well-approximate the distribution of consumer types for this problem. In practice we may also want to try multiple starting values for each set of draws to be sure that optimization is working properly.

### 2. Use scrambled Halton sequences

Now let's try using scrambled Halton draws to create the nodes. We'll still use a random number generator to draw from the demographics data.

In [21]:
for draws in [10, 100, 500, 1_000, 2_000]:
    print(f"Using {draws} ...")
    halton_data = demographic_data[['market_ids', 'log_income']].groupby('market_ids', as_index=False).sample(n=draws, replace=True, random_state=0)
    halton_integration = pyblp.Integration('halton', size=len(halton_data), specification_options={'seed': 0})
    halton_data['nodes0'] = pyblp.build_integration(halton_integration, dimensions=1).nodes.flatten()
    halton_data['weights'] = 1 / draws
    agent_results.append(solve(f"{draws} Halton Draws", halton_data))

pd.DataFrame(agent_results)

Using 10 ...
Using 100 ...
Using 500 ...
Using 1000 ...
Using 2000 ...


,Specification,Time,Converged,Gradient Norm,Sigma on Price,Pi on Mushy,Pi on Price,Alpha
0,10 Monte Carlo Draws,2.961,True,1.425e-09,7.271,0.127,-7.085,22.752
1,100 Monte Carlo Draws,3.604,True,1.123e-06,5.748,0.104,-5.728,12.020
2,500 Monte Carlo Draws,8.926,True,8.619e-09,6.064,0.108,-6.054,14.487
3,1000 Monte Carlo Draws,13.725,True,2.120e-07,6.141,0.109,-5.956,13.664
4,2000 Monte Carlo Draws,18.678,True,2.116e-07,6.051,0.108,-5.989,13.971
5,10 Halton Draws,2.734,True,6.529e-09,7.851,0.129,-6.506,16.979
6,100 Halton Draws,3.437,True,2.083e-07,6.204,0.101,-5.510,10.104
7,500 Halton Draws,8.235,True,4.225e-08,6.077,0.108,-5.955,13.700
8,1000 Halton Draws,11.405,True,8.532e-09,6.014,0.108,-5.981,13.904
9,2000 Halton Draws,21.938,True,1.654e-09,6.028,0.108,-5.981,13.907


Again, estimates seem to stabilize by 1,000 draws. But estimates are closer to the 1,000 estimates with fewer draws, indicating that we're getting more out of fewer scrambled Halton draws.

### 3. Try using quadrature

Let's first construct a two-dimensional product rule of quadrature nodes and weights.

In [22]:
quad_data = pd.DataFrame(pyblp.data_to_dict(pyblp.build_integration(pyblp.Integration('product', size=7), dimensions=2)))
quad_data

,weights,nodes0,nodes1
0,3.006e-07,-3.750,-3.750
1,1.686e-05,-3.750,-2.367
2,1.317e-04,-3.750,-1.154
3,2.506e-04,-3.750,-0.000
4,1.317e-04,-3.750,1.154
5,1.686e-05,-3.750,2.367
6,3.006e-07,-3.750,3.750
7,1.686e-05,-2.367,-3.750
8,9.460e-04,-2.367,-2.367
9,7.385e-03,-2.367,-1.154


Now let's use the market-level log income means and standard deviations from above to transform the first dimension into normally distributed log income draws, leaving the second column as standard normal unobserved preference heterogenity.

In [23]:
quad_data = quad_data.merge(demographic_variation, how='cross')
quad_data['log_income'] = quad_data['log_income_mean'] + quad_data['log_income_std'] * quad_data['nodes1']
quad_data = quad_data[['market_ids', 'log_income', 'nodes0', 'weights']]
quad_data

,market_ids,log_income,nodes0,weights
0,C01Q1,5.084,-3.75,3.006e-07
1,C01Q2,5.084,-3.75,3.006e-07
2,C03Q1,1.856,-3.75,3.006e-07
3,C03Q2,1.856,-3.75,3.006e-07
4,C04Q1,6.502,-3.75,3.006e-07
...,...,...,...,...
4601,C61Q2,11.073,3.75,3.006e-07
4602,C63Q1,11.158,3.75,3.006e-07
4603,C63Q2,11.158,3.75,3.006e-07
4604,C65Q1,11.125,3.75,3.006e-07


Let's see how estimates differ.

In [24]:
agent_results.append(solve(f"{7**2} Quadrature Nodes", quad_data))
pd.DataFrame(agent_results)

,Specification,Time,Converged,Gradient Norm,Sigma on Price,Pi on Mushy,Pi on Price,Alpha
0,10 Monte Carlo Draws,2.961,True,1.425e-09,7.271,0.127,-7.085,22.752
1,100 Monte Carlo Draws,3.604,True,1.123e-06,5.748,0.104,-5.728,12.020
2,500 Monte Carlo Draws,8.926,True,8.619e-09,6.064,0.108,-6.054,14.487
3,1000 Monte Carlo Draws,13.725,True,2.120e-07,6.141,0.109,-5.956,13.664
4,2000 Monte Carlo Draws,18.678,True,2.116e-07,6.051,0.108,-5.989,13.971
5,10 Halton Draws,2.734,True,6.529e-09,7.851,0.129,-6.506,16.979
6,100 Halton Draws,3.437,True,2.083e-07,6.204,0.101,-5.510,10.104
7,500 Halton Draws,8.235,True,4.225e-08,6.077,0.108,-5.955,13.700
8,1000 Halton Draws,11.405,True,8.532e-09,6.014,0.108,-5.981,13.904
9,2000 Halton Draws,21.938,True,1.654e-09,6.028,0.108,-5.981,13.907


Results are similar to the Monte Carlo and Halton configurations with many nodes, but optimization was far faster.

### 5. Approximate the optimal instruments

First let's approximate the optimal instruments and re-create the problem.

In [25]:
optimal_iv_results = rc_results.compute_optimal_instruments()
optimal_problem = optimal_iv_results.to_problem()
optimal_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     2     1    4     2  

Formulations:
       Column Indices:             0         1   
-----------------------------  ----------  ------
 X1: Linear Characteristics      prices          
X2: Nonlinear Characteristics    mushy     prices
       d: Demographics         log_income        

Now let's re-solve the problem.

In [26]:
pyblp.options.verbose = True
optimal_problem.solve(
    sigma=[
        [0, 0],
        [0, 6],
    ], 
    pi=[
        [0.2],
        [-6],
    ],
    method='1s', 
    optimization=optimization,
)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy   +2.00E-01 
prices  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.00E+00  |  prices    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     +INF    
prices  +0.00E+00    +INF     |  prices    +INF    

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective     Projected                                   
Step     Time       Iterations   Evaluations  Iterations   

Results are similar, although the amount of unobserved preference heterogeneity for prices is estimated to be somewhat larger, with a smaller standard error. The fact that results are similar suggests that the non-optimal instruments we've been using are fairly close to the optimal ones. Perhaps our instrument for targeting $\Sigma$ has some nontrivial differences.

### 5. Add another instrument and compute the optimal weighting matrix

Let's add a second instrument to target the parameter in $\Pi$ on log income and mushy, but using cross-market variation in the standard deviation of income.

In [27]:
product_data = product_data.merge(demographic_variation[['market_ids', 'log_income_std']], on='market_ids')
product_data['demand_instruments4'] = product_data['log_income_std'] * product_data['demand_instruments3']
overidentified_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
overidentified_problem

Dimensions:
 T    N      I     K1    K2    D    MD    ED 
---  ----  -----  ----  ----  ---  ----  ----
94   2256  94000   1     2     1    5     2  

Formulations:
       Column Indices:             0         1   
-----------------------------  ----------  ------
 X1: Linear Characteristics      prices          
X2: Nonlinear Characteristics    mushy     prices
       d: Demographics         log_income        

Now let's re-run the first stage with the extra instrument.

In [28]:
pyblp.options.verbose = True
overidentified_results = overidentified_problem.solve(
    sigma=[
        [0, 0],
        [0, 6],
    ], 
    pi=[
        [0.2],
        [-6],
    ], 
    method='1s', 
    optimization=optimization,
)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy   +2.00E-01 
prices  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.00E+00  |  prices    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     +INF    
prices  +0.00E+00    +INF     |  prices    +INF    

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective     Projected                                   
Step     Time       Iterations   Evaluations  Iterations   

Since we're overidentified, the objective at the optimum is no longer essentially zero. But our first- and second-order conditions pan out, so we are likely at the global optimum. In practice we would want to try multiple starting values.

Results are similar, although the parameter the new instrument targets is now estimated to be slightly dampened, with the parameter in $\Sigma$ on prices a bit larger to pick up the extra unobserved preference heterogeneity.

Now let's run a second step with the optimal weighting matrix.

In [29]:
pyblp.options.verbose = True
overidentified_problem.solve(
    sigma=[
        [0, 0],
        [0, 6],
    ], 
    pi=[
        [0.2],
        [-6],
    ], 
    method='1s', 
    optimization=optimization,
    W=overidentified_results.updated_W,
)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy   +2.00E-01 
prices  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.00E+00  |  prices    -INF    

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     +INF    
prices  +0.00E+00    +INF     |  prices    +INF    

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped  Objective   Objective     Projected                                   
Step     Time       Iterations   Evaluations  Iterations   

Results are fairly similar to the first step. Standard errors are slightly smaller.

### 6. Incorporate supply-side restrictions into estimation

Let's set up the problem with a simple supply side.

In [30]:
product_data['firm_ids'] = product_data['product_ids'].str[:2]
supply_product_formulations = (
    pyblp.Formulation('0 + prices', absorb='C(market_ids) + C(product_ids)'), 
    pyblp.Formulation('0 + mushy + prices'),
    pyblp.Formulation('1 + mushy'),
)
supply_problem = pyblp.Problem(supply_product_formulations, product_data, agent_formulation, agent_data)
supply_problem

Dimensions:
 T    N     F     I     K1    K2    K3    D    MD    MS    ED 
---  ----  ---  -----  ----  ----  ----  ---  ----  ----  ----
94   2256   5   94000   1     2     2     1    5     2     2  

Formulations:
        Column Indices:              0         1   
-------------------------------  ----------  ------
  X1: Linear Characteristics       prices          
 X2: Nonlinear Characteristics     mushy     prices
X3: Linear Cost Characteristics      1       mushy 
        d: Demographics          log_income        

We can solve it almost like before. The only difference is that $\alpha$ is now a parameter that needs to be optimized over because it affects demand elasticities, which in turn are needed to impute marginal costs from pricing optimality. So we'll need to specify an initial value for it.

In [31]:
pyblp.options.verbose = True
supply_problem.solve(
    sigma=[
        [0, 0],
        [0, 6],
    ], 
    pi=[
        [0.2],
        [-6],
    ],
    beta=10,
    method='1s', 
    optimization=optimization,
)
pyblp.options.verbose = False

Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy   +2.00E-01 
prices  +0.00E+00  +6.00E+00  |  prices  -6.00E+00 

Beta Initial Values:
 prices  
---------
+1.00E+01

Nonlinear Coefficient Lower Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     -INF    
prices  +0.00E+00  +0.00E+00  |  prices    -INF    

Beta Lower Bounds:
 prices  
---------
  -INF   

Nonlinear Coefficient Upper Bounds:
Sigma:    mushy     prices    |   Pi:    log_income
------  ---------  ---------  |  ------  ----------
mushy   +0.00E+00             |  mushy     +INF    
prices  +0.00E+00    +INF     |  prices    +INF    

Beta Upper Bounds:
 prices  
---------
  +INF   

Starting optimization ...

GMM   Computation  Optimization   Objective   Fixed Point  

Supply-side estimates suggest that producing mushy cereal is less costly than other types. Demand-side estimates are the same as before because we are using the same moments to pin them down.